In [0]:
!pip install pytorch-pretrained-bert
!pip install transformers


     |████████████████████████████████| 133kB 2.8MB/s 
     |████████████████████████████████| 645kB 8.6MB/s 
     |████████████████████████████████| 368kB 2.7MB/s 
     |████████████████████████████████| 1.0MB 58.2MB/s 
     |████████████████████████████████| 860kB 63.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=12950fb0fb78b427012c8735c63ad2db89ad136673d5d652335c41b6f9244b3e
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [0]:
import numpy as np
import torch
from typing import List
import torch.nn as nn
from torch.nn import init
import torch.optim as optim
import math
import random
import os
import csv
import sys
import tqdm
from tqdm import tqdm, trange
from pathlib import Path 
import time
from tqdm import tqdm
import json
from gensim.models import Word2Vec
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
# from pytorch_pretrained_bert import BertModel, BertForMaskedLM, BertForMultipleChoice 
from transformers import BertConfig, BertModel, BertForMaskedLM, BertForMultipleChoice
from transformers import PreTrainedTokenizer, BertTokenizer
from transformers import AdamW, get_linear_schedule_with_warmup
from typing import List
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule

# from tqdm import tqdm_notebook as tqdm

Using TensorFlow backend.


In [0]:
#!pip install pytorch-pretrained-bert

In [0]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

In [0]:
# XXX: only one GPU on Colab and isn’t guaranteed
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import GPUtil as GPU
GPUs = GPU.getGPUs()
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7410 sha256=49c23d01c6a6ec81187d36b5e0c532efa2824e477332bacc454579df6f6e48bf
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 24.7 GB  | Proc size: 2.7 GB
GPU RAM Free: 15567MB | Used: 713MB | Util   4% | Total 16280MB


In [0]:
# !kill -9 -1

In [0]:
#@title Default title text
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#@title Default title text
# Task we are doing (ROCStory next sentence prediction)
processors = {
    "ROCStory": ROCStoryProcessor()
}


In [0]:
#@title Default title text
class DataProcessor(object):
    """Base class for data converters for multiple choice data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_test_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the test set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

In [0]:
#@title Default title text
class InputExample(object):
    """A single training/test example for multiple choice"""

    def __init__(self, example_id, contexts, endings, label=None):
        """Constructs a InputExample.
        Args:
            example_id: Unique id for the example.
            contexts: list of str. The untokenized text of the first sequence (context of corresponding question).
            question: string. The untokenized text of the second sequence (question).
            endings: list of str. multiple choice's options. Its length must be equal to contexts' length.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.example_id = example_id
        # self.question = question
        self.contexts = contexts
        self.endings = endings
        self.label = label

In [0]:
#@title Default title text
class InputFeatures(object):
    def __init__(self,
                 example_id,
                 choices_features,
                 label

    ):
        self.example_id = example_id
        self.choices_features = [
            {
                'input_ids': input_ids,
                'input_mask': input_mask,
                'segment_ids': segment_ids
            }
            for input_ids, input_mask, segment_ids in choices_features
        ]
        self.label = label

In [0]:
#@title Default title text
def _create_examples(lines: List[List[str]]):
        """Creates examples for the training and dev sets."""
        # if type == "train" and lines[0][-1] != 'label':
        #     raise ValueError(
        #         "For training, the input file must contain a label column."
        #     )

        # for line in lines[1:]:
        #   print(line)
        # print(lines[4])
          

        examples = [
            InputExample(
                example_id= line[0],
                # question="",   #line[2],  # in the swag dataset, the
                # common beginning of each
                # choice is stored in "sent2".
                contexts = [line[1], line[2], line[3], line[4]],
                endings = [line[5], line[6]],   
                label= line[7]
            ) for line in lines[1:]  # we skip the line with the column names
        ]

        return examples

In [0]:
#@title Default title text
class ROCStoryProcessor(DataProcessor):
    """Processor for the SWAG data set."""

    def get_train_examples(self, data_dir):
        """See base class."""
        # logger.info("LOOKING AT {} train".format(data_dir))
        return _create_examples(ROCStoryProcessor()._read_csv(data_dir))
        # return ._read_csv(data_dir)

    def get_dev_examples(self, data_dir):
        """See base class."""
        # logger.info("LOOKING AT {} dev".format(data_dir))
        return self._create_examples(self._read_csv(os.path.join(data_dir, "val.csv")), "dev")

    def get_test_examples(self, data_dir):
        """See base class."""
        # logger.info("LOOKING AT {} dev".format(data_dir))
        raise ValueError(
            "For swag testing, the input file does not contain a label column. It can not be tested in current code"
            "setting!"
        )
        return self._create_examples(self._read_csv(os.path.join(data_dir, "test.csv")), "test")
    def get_labels(self,):
        """See base class."""
        return ["1", "2"]

    def _read_csv(self, input_file):
        
        with open(input_file, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            lines = []
            
            for line in reader:
                if sys.version_info[0] == 2:
                    line = list(unicode(cell, 'utf-8') for cell in line)
                lines.append(line)
            return lines
# obj = ROCStoryProcessor()
# obj.get_train_examples(path_train)

In [0]:
#@title Default title text
def load_and_cache_examples(data_dir, task, tokenizer, evaluate=False, test=False):
    # if args.local_rank not in [-1, 0]:
    #     torch.distributed.barrier()  # Make sure only the first process in distributed training process the dataset, and the others will use the cache

    processor = processors[task]
    max_seq_length = 128
    # Load data features from cache or dataset file
    # if evaluate:
    #     cached_mode = 'dev'
    # elif test:
    #     cached_mode = 'test'
    # else:
    #     cached_mode = 'train'
    # assert (evaluate == True and test == True) == False
    # cached_features_file = os.path.join(data_dir, 'cached_{}_{}_{}_{}'.format(
    #     cached_mode,
    #     list(filter(None, args.model_name_or_path.split('/'))).pop(),
    #     str(args.max_seq_length),
    #     str(task)))
    # if os.path.exists(cached_features_file) and not args.overwrite_cache:
    #     logger.info("Loading features from cached file %s", cached_features_file)
    #     features = torch.load(cached_features_file)
    
    # logger.info("Creating features from dataset file at %s", args.data_dir)
    label_list = processor.get_labels()
    if evaluate:
        examples = processor.get_dev_examples(data_dir)
    elif test:
        examples = processor.get_test_examples(data_dir)
    else:
        examples = processor.get_train_examples(data_dir)
    # logger.info("Training number: %s", str(len(examples)))
    # print(examples)
    features = convert_examples_to_features(
        examples,
        label_list,
        max_seq_length,
        PreTrainedTokenizer,
        pad_on_left=bool(0),                 # pad on the left for xlnet
        pad_token_segment_id= 0
    )
        # if args.local_rank in [-1, 0]:
        #     logger.info("Saving features into cached file %s", cached_features_file)
        #     torch.save(features, cached_features_file)

    # if args.local_rank == 0:
    #     torch.distributed.barrier()  # Make sure only the first process in distributed training process the dataset, and the others will use the cache

    # Convert to Tensors and build dataset
    all_input_ids = torch.tensor(select_field(features, 'input_ids'), dtype=torch.long)
    all_input_mask = torch.tensor(select_field(features, 'input_mask'), dtype=torch.long)
    all_segment_ids = torch.tensor(select_field(features, 'segment_ids'), dtype=torch.long)
    all_label_ids = torch.tensor([f.label for f in features], dtype=torch.long)

    dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
    return dataset

In [0]:
#@title Default title text
def convert_examples_to_features(
    examples: List[InputExample],
    label_list: List[str],
    max_length: int,
    tokenizer = PreTrainedTokenizer,
    pad_token_segment_id=0,
    pad_on_left=False,
    pad_token=0,
    mask_padding_with_zero=True,
) -> List[InputFeatures]:
    """
    Loads a data file into a list of `InputFeatures`
    """
    
    label_map = {label : i for i, label in enumerate(label_list)}

    features = []
    for (ex_index, example) in tqdm(enumerate(examples), desc="convert examples to features"):
        # if ex_index % 10000 == 0:
            # logger.info("Writing example %d of %d" % (ex_index, len(examples)))
        choices_features = []
        for ending_idx, (context, ending) in enumerate(zip(example.contexts, example.endings)):
            text_a = context
            # if example.question.find("_") != -1:
            #     # this is for cloze question
            #     text_b = example.question.replace("_", ending)
            # else:
            #     text_b = example.question + " " + ending

            text_b = ending
            print(text_a)
            print(text_b)
            inputs = tokenizer.encode_plus(
                text_a,
                text_b,
                add_special_tokens=True,
                max_length=max_length,
            )
            # if 'num_truncated_tokens' in inputs and inputs['num_truncated_tokens'] > 0:
            #     logger.info('Attention! you are cropping tokens (swag task is ok). '
            #             'If you are training ARC and RACE and you are poping question + options,'
            #             'you need to try to use a bigger max seq length!')

            
            input_ids, token_type_ids = inputs["input_ids"], inputs["token_type_ids"]

            # The mask has 1 for real tokens and 0 for padding tokens. Only real
            # tokens are attended to.
            attention_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

            # Zero-pad up to the sequence length.
            padding_length = max_length - len(input_ids)
            if pad_on_left:
                input_ids = ([pad_token] * padding_length) + input_ids
                attention_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + attention_mask
                token_type_ids = ([pad_token_segment_id] * padding_length) + token_type_ids
            else:
                input_ids = input_ids + ([pad_token] * padding_length)
                attention_mask = attention_mask + ([0 if mask_padding_with_zero else 1] * padding_length)
                token_type_ids = token_type_ids + ([pad_token_segment_id] * padding_length)

            assert len(input_ids) == max_length
            assert len(attention_mask) == max_length
            assert len(token_type_ids) == max_length
            choices_features.append((input_ids, attention_mask, token_type_ids))


        label = label_map[example.label]
        print(choices_features)

        if ex_index < 2:
            logger.info("*** Example ***")
            logger.info("race_id: {}".format(example.example_id))
            for choice_idx, (input_ids, attention_mask, token_type_ids) in enumerate(choices_features):
                logger.info("choice: {}".format(choice_idx))
                logger.info("input_ids: {}".format(' '.join(map(str, input_ids))))
                logger.info("attention_mask: {}".format(' '.join(map(str, attention_mask))))
                logger.info("token_type_ids: {}".format(' '.join(map(str, token_type_ids))))
                logger.info("label: {}".format(label))

        features.append(
            InputFeatures(
                example_id=example.example_id,
                choices_features=choices_features,
                label=label,
            )
        )

    return features

In [0]:
#@title Default title text
unk = '<UNK>'
# Consult the PyTorch documentation for information on the functions used below:
# https://pytorch.org/docs/stable/torch.html

class RNN(nn.Module):
	def __init__(self, input_dim, h1, h):
		super(RNN, self).__init__()
		self.input_dim = input_dim
		self.h = h
		self.h1 = h1
		# self.rnn = nn.RNNCell(self.input_dim, self.h1)
		self.rnn = nn.RNN(input_size=self.input_dim, hidden_size=self.h1, num_layers=1, nonlinearity = 'relu')
		self.W1 = nn.Linear(self.h1, self.h)
		self.W2 = nn.Linear(self.h, 5)
	
		self.activation = nn.ReLU()		
		self.softmax = nn.LogSoftmax()
		self.loss = nn.NLLLoss()

	def compute_Loss(self, predicted_vector, gold_label):
		return self.loss(predicted_vector, gold_label)	
	
	def init_hidden(self):
        # (num_layers, batch_size, n_neurons)
			h = torch.randn(1, 1, self.h1, requires_grad = True, dtype=torch.float)*0.01
			return (h.cuda())

	def forward(self, input_matrix): 
		n_word = input_matrix.size(0)
		n_embed = input_matrix.size(1)
		input_matrix = input_matrix.reshape(n_word, 1, n_embed)
		self.hx = self.init_hidden()
	  
		rnn_out, self.hx = self.rnn(input_matrix, self.hx)     # Explain the architecture.
		rnn_out = rnn_out.reshape(n_word, self.h1)
		self.hx = self.hx.squeeze(0).squeeze(1)	  
		h_out = self.hx
		z1 = self.W1(h_out)
		z3 = self.activation(z1)
		z2 = self.W2(z3)	
		predicted_vector = self.softmax(self.activation(z2))		# is activation required ?
		return predicted_vector

In [0]:
#@title Default title text
class BERT(nn.Module):
  def __init__(self):
    super(BERT, self).__init__()
    # self.h = h
    self.bert = BertForMultipleChoice.from_pretrained("bert-base-uncased")
    # self.W1 = nn.Linear(self.h, 2)
  
    # self.activation = nn.ReLU()		
    # self.softmax = nn.LogSoftmax()
    self.loss = nn.NLLLoss()

  def compute_Loss(self, predicted_vector, gold_label):
    return self.loss(predicted_vector, gold_label)
  
  def forward(self, story, att_mask, label):
    input_ids = story.unsqueeze(0)
    att_mask = att_mask.unsqueeze(0)
    label = label.unsqueeze(0)
    loss = self.bert(input_ids, token_type_ids=None, attention_mask=att_mask, labels = label)
    # cont_vect2, _ = self.bert(story2, token_type_ids=None, attention_mask=att_mask2)

    return loss
    


In [0]:
# Load all files into DataFrame(Pandas)
# path_train = '/content/drive/My Drive/p4_NLP/train.csv'
# path_val = '/content/drive/My Drive/p4_NLP/dev.csv'
# path_test = '/content/drive/My Drive/p4_NLP/test.csv'
path_train = 'train.csv'
path_val = 'dev.csv'
path_test = 'test.csv'
data_train = pd.read_csv(path_train)
data_val = pd.read_csv(path_val)
data_test =pd.read_csv(path_test)
data_test.head(2)


,InputStoryid,InputSentence1,InputSentence2,InputSentence3,InputSentence4,RandomFifthSentenceQuiz1,RandomFifthSentenceQuiz2
0,b929f263-1dcd-4a0b-b267-5d5ff2fe65bb,My friends all love to go to the club to dance.,They think it's a lot of fun and always invite.,I finally decided to tag along last Saturday.,I danced terribly and broke a friend's toe.,My friends decided to keep inviting me out as ...,"The next weekend, I was asked to please stay h..."
1,7cbbc0af-bcce-4f56-871d-963f9bb6a99d,I tried going to the park the other day.,The weather seemed nice enough for a walk.,Within minutes of getting there I started snee...,My eyes were watery and it was hard to breathe.,My allergies were too bad and I had to go back...,It reminded me of how much I loved spring flow...


In [0]:
# !unzip '/content/drive/My Drive/p4_NLP/glove.840B.300d.zip'

In [0]:
def fetch_data(data):
	token_data = []
	labels = []

	for item in data.iterrows():
		
		# story = '[CLS]' + ' ' + item[1][1] + ' ' + item[1][2] + ' ' + item[1][3] + ' ' + item[1][4]
		story = item[1][1] + ' ' + item[1][2] + ' ' + item[1][3] + ' ' + item[1][4]
		
		# # Manually adding CLS --> SEP --> SEP
		# storyend1  = story + ' ' + item[1][5] + ' ' + '[SEP]'
		# storyend2 = story + ' ' + item[1][6] + ' ' + '[SEP]'
		
		# Adding CLS --> SEP --> SEP 
		storyend1  = story + 'SEP' + item[1][5] 
		storyend2 = story + 'SEP' + item[1][6] 
		
		# # Adding CLS --> SEP
		# storyend1  = story + ' ' + item[1][5] 
		# storyend2 = story + ' ' + item[1][6]
		
		
		token_data.append(storyend1)
		token_data.append(storyend2)
		labels.append(item[1][7])      # corresponding to label
	  
	return token_data, labels
	

In [0]:
def fetch_data_test(data):
    token_data = []
    ids = []
    labels = []
    for item in data.iterrows():
      story = item[1][1] + ' ' + item[1][2] + ' ' + item[1][3] + ' ' + item[1][4]
      storyend1  = story + 'SEP' + item[1][5] 
      storyend2 = story + 'SEP' + item[1][6] 
      token_data.append(storyend1)
      token_data.append(storyend2)
      ids.append(item[1][0])
      labels.append(1)
    
    return token_data, ids, labels


In [0]:
from numpy import *
import math
import matplotlib.pyplot as plt
def plot_learning_curve(train_loss_list, val_acc_list, epoch):
  fig = plt.figure()
  epoch_axis = linspace(1, epoch, epoch)
  a, = plt.plot(epoch_axis,train_loss_list, 'r*-', label='Line 1')
  b, = plt.plot(epoch_axis,val_acc_list, 'b*-', label='Line 2')
  plt.legend([a, b], ['Training Loss', 'Validation Acc'], loc='best')
  plt.xlabel('Epochs')
  plt.ylabel('Performance Parameters')
  plt.title('Learning Curve')
  plt.show()

# plot_learning_curve([1,2,3], [4,5,6], 3)


In [0]:
def main():
     
     best_model_file_name = 'bestmodel_bert.pt'
     model_choice = 'bert-large-uncased'   # 'bert-large-cased'
     random.seed(1234)
     np.random.seed(1234)
     torch.manual_seed(1234)
     if device_name == '/device:GPU:0':
       torch.cuda.manual_seed_all(1234)
    
     print("Fetching data...")
     train_data, train_labels = fetch_data(data_train)        # 
     val_data, val_labels = fetch_data(data_val)              #
     test_data, ids, test_labels = fetch_data_test(data_test)
     print("Fetched and extracted data!")
     tokenizer = BertTokenizer.from_pretrained(model_choice, do_lower_case=True)
    #  tokenizer = PreTrainedTokenizer()
     
     input_ids_train_data = []
     token_typ_ids_train_data = []
     input_ids_val_data = []
     token_typ_ids_val_data = []
     input_ids_test_data = []
     token_typ_ids_test_data = []

    #-------------------------------------------------------
    # Training with CLS --> SEP ---> SEP
     for train in train_data:
       item_train = train.split('SEP')
       output_train = tokenizer.encode_plus(item_train[0], text_pair=item_train[1], add_special_tokens=True)
       input_ids_train_data.append(output_train['input_ids'])
       token_typ_ids_train_data.append(output_train['token_type_ids'])
          
     for val in val_data:
       item_val = val.split('SEP')
       output_val = tokenizer.encode_plus(item_val[0], text_pair=item_val[1], add_special_tokens=True)
       input_ids_val_data.append(output_val['input_ids'])
       token_typ_ids_val_data.append(output_val['token_type_ids'])
     
     for test in test_data:
       item_test = test.split('SEP')
       output_test = tokenizer.encode_plus(item_test[0], text_pair=item_test[1], add_special_tokens=True)
       input_ids_test_data.append(output_test['input_ids'])
       token_typ_ids_test_data.append(output_test['token_type_ids'])
    #-------------------------------------------------------

    # # Training with CLS --> SEP
    #  for train in train_data:
    #    output_train = tokenizer.encode_plus(train, add_special_tokens=True)
    #    input_ids_train_data.append(output_train['input_ids'])
    #    token_typ_ids_train_data.append(output_train['token_type_ids'])
          
    #  for val in val_data:
    #    output_val = tokenizer.encode_plus(val, add_special_tokens=True)
    #    input_ids_val_data.append(output_val['input_ids'])
    #    token_typ_ids_val_data.append(output_val['token_type_ids'])

        
    #  tokenized_train_data = [tokenizer.tokenize(sent) for sent in train_data]
    #  tokenized_val_data = [tokenizer.tokenize(sent) for sent in val_data]
    #  print("Tokenized Data!")
     
     MAX_LEN = 80
     input_ids_train_data = pad_sequences(input_ids_train_data, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")    # imported from Keras Preprocessing
     input_ids_val_data = pad_sequences(input_ids_val_data, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")        # imported from Keras Preprocessing
     input_ids_test_data = pad_sequences(input_ids_test_data, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
     token_typ_ids_train_data = pad_sequences(token_typ_ids_train_data, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
     token_typ_ids_val_data = pad_sequences(token_typ_ids_val_data, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
     token_typ_ids_test_data = pad_sequences(token_typ_ids_test_data, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
     
     print("Input IDs and padding generated!")
     att_mask_train_data = []
     att_mask_val_data = []
     att_mask_test_data = []
     for seq in input_ids_train_data:
          seq_mask = [float(i>0) for i in seq]
          att_mask_train_data.append(seq_mask)
     for seq in input_ids_val_data:
          seq_mask = [float(i>0) for i in seq]
          att_mask_val_data.append(seq_mask)   
     for seq in input_ids_test_data:
          seq_mask = [float(i>0) for i in seq]
          att_mask_test_data.append(seq_mask)   
     print("Attention Masks generated!")  
    
     print("Total training documents #:", int(len(input_ids_train_data)/2))
     print("Total Validation documents #:", int(len(att_mask_val_data)/2))
     print("Total Test documents #:", int(len(att_mask_test_data)/2))
     print("Max sequence Length after padding:", len(input_ids_train_data[0]))

     n_train = int(len(input_ids_train_data)/2)
     n_val = int(len(att_mask_val_data)/2)
     n_test = int(len(att_mask_test_data)/2)
     n_seq_pad = len(input_ids_train_data[0])
  
     input_ids_train_data = torch.tensor(input_ids_train_data).long()
     input_ids_val_data = torch.tensor(input_ids_val_data).long()
     input_ids_test_data = torch.tensor(input_ids_test_data).long()
     att_mask_train_data = torch.tensor(att_mask_train_data).long()
     att_mask_val_data = torch.tensor(att_mask_val_data).long()
     att_mask_test_data = torch.tensor(att_mask_test_data).long()
     token_typ_ids_train_data = torch.tensor(token_typ_ids_train_data).long()
     token_typ_ids_val_data = torch.tensor(token_typ_ids_val_data).long()
     token_typ_ids_test_data = torch.tensor(token_typ_ids_test_data).long()
     train_labels = torch.tensor(train_labels) - 1
     val_labels = torch.tensor(val_labels) - 1
     test_labels = torch.tensor(test_labels)
     
     
     input_ids_train_data = input_ids_train_data.reshape(n_train, 2, n_seq_pad)
     input_ids_val_data = input_ids_val_data.reshape(n_val, 2, n_seq_pad)
     input_ids_test_data = input_ids_test_data.reshape(n_test, 2, n_seq_pad)
     att_mask_train_data = att_mask_train_data.reshape(n_train, 2, n_seq_pad)
     att_mask_val_data = att_mask_val_data.reshape(n_val, 2, n_seq_pad)
     att_mask_test_data = att_mask_test_data.reshape(n_test, 2, n_seq_pad)
     token_typ_ids_train_data = token_typ_ids_train_data.reshape(n_train, 2, n_seq_pad)
     token_typ_ids_val_data = token_typ_ids_val_data.reshape(n_val, 2, n_seq_pad)
     token_typ_ids_test_data = token_typ_ids_test_data.reshape(n_test, 2, n_seq_pad)
           
     #  config = BertConfig.from_pretrained('bert-base-uncased', num_labels=2)
     model = BertForMultipleChoice.from_pretrained(model_choice)
     model.cuda()
     for p in model.parameters():
         p.requires_grad = True
    
    # # -------------------------------------------------------------------------------
    # # TEST PHASE (SUCCESS!) # NEED TO IMPLEMENT FANCY SCHEDULING!
    # #  a = model(input_ids_train_data[0], att_mask_train_data[0], train_labels[0])
    # #  print(a)
    # #  predicted_vector = a[1].squeeze(0)
    # #  print(predicted_vector.view(1,-1))
    # #  print(train_labels[0].unsqueeze(0))
    # #  loss = model.compute_Loss(predicted_vector.view(1,-1), train_labels[0].unsqueeze(0))
    # #  print(loss)
    # # -------------------------------------------------------------------------------
    
     # Inputing the data into DataLoader for training and Validation
     batch_size = 8
     train_set = TensorDataset(input_ids_train_data, att_mask_train_data, token_typ_ids_train_data, train_labels)
     train_sampler = RandomSampler(train_set)
     val_set = TensorDataset(input_ids_val_data, att_mask_val_data ,token_typ_ids_val_data, val_labels)
     val_sampler = SequentialSampler(val_set)
     test_set = TensorDataset(input_ids_test_data, att_mask_test_data ,token_typ_ids_test_data, test_labels)
     train_loader = DataLoader(train_set, batch_size = batch_size, sampler = train_sampler)
     val_loader =  DataLoader(val_set, batch_size = batch_size, shuffle = False)
     test_loader = DataLoader(test_set, batch_size = batch_size, shuffle = False)

     # Prepare optimizer and schedule (linear warmup and decay)
     no_decay = ['bias', 'LayerNorm.weight']
     n_epoch = 3
     learning_rate = 5e-5
     adam_epsilon = 1e-8
     warmup_steps = 4
     t_total = len(train_loader) // batch_size * n_epoch
     weight_decay = 0.0
     optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': weight_decay},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
     optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=adam_epsilon)
     scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total)
     
     
    #  torch.cuda.empty_cache()
     best_val_acc = 0
     train_loss_list = []
     val_acc_list = []
     for epoch in tqdm(range(n_epoch), position=0, leave=False):
       model.train()
       loss = None
       correct = 0
       total = 0
       loss_epoch = 0
       start_time = time.time()
       for step, batch in enumerate(train_loader):
         optimizer.zero_grad()
         loss = None
         batch = tuple(t.to(device) for t in batch)
         input_ids, att_mask, token_id_typ, label = batch
         loss_batch, pred_vector = model(input_ids,  token_type_ids=token_id_typ, attention_mask=att_mask, labels = label)
         pred_label = torch.argmax(pred_vector, axis=1)
         correct += torch.sum(pred_label == label).item()
         total += input_ids.size(0)  
         loss = loss_batch
         loss_epoch += loss_batch.item()
         loss.backward()
         optimizer.step()
         scheduler.step()

       train_loss_list.append(loss_epoch) 
      #  print(train_loss_list)   
       print("\nTraining completed for epoch {}".format(epoch + 1))
       print("Training accuracy for epoch {}: {}".format(epoch + 1, correct / total))
       print("Training time for this epoch: {}".format(time.time() - start_time))
       print("Total number of training examples: {}".format(total))
       print("Number of correctly predicted examples: {}".format(correct))

       with torch.no_grad():
         model.eval()
         loss_val = None
         correct = 0
         total = 0
         total_pred_label = torch.tensor([0]).cuda()
         start_time = time.time()
         for step, batch in enumerate(val_loader):
           
           batch = tuple(t.to(device) for t in batch)
           input_ids, att_mask, token_id_typ, label = batch
           loss_batch, pred_vector = model(input_ids,  token_type_ids=token_id_typ, attention_mask=att_mask, labels = label)
           pred_label = torch.argmax(pred_vector, axis=1)
           total_pred_label = torch.cat((total_pred_label, (pred_label+1)), axis=0)
           correct += torch.sum(pred_label == label).item()
           total += input_ids.size(0)
           loss_val = loss_batch
         
         total_pred_label = total_pred_label[1:]
         total_pred_label = total_pred_label.detach().cpu().numpy()
         total_pred_label = total_pred_label.tolist()
         output = pd.DataFrame({'Prediction': total_pred_label})
         path = 'val_part_b.csv'
         output.to_csv(path, index = False)
         output.head(5)
          
         print("\nTraining completed for epoch {}".format(epoch + 1))
         print("Validation accuracy for epoch {}: {}".format(epoch + 1, correct / total))
         print("Validation time for this epoch: {}".format(time.time() - start_time))
         print("Total number of Validation examples: {}".format(total))
         print("Number of correctly predicted examples: {}".format(correct)) 

         epoch_val_acc = correct / total
         val_acc_list.append(epoch_val_acc)
         if epoch_val_acc > best_val_acc:
           torch.save(model.state_dict(), best_model_file_name)
           best_val_acc = epoch_val_acc


     model.load_state_dict(torch.load(best_model_file_name))
     model.optimizer = optimizer

          
     # Plot the performance curve for the model
     plot_learning_curve(train_loss_list, val_acc_list, n_epoch)  
     
     start_time = time.time()
     total_pred_label = torch.tensor([0]).cuda()
     for step, batch in enumerate(test_loader):
       batch = tuple(t.to(device) for t in batch)
       input_ids, att_mask, token_id_typ, label = batch
       _, pred_vector = model(input_ids,  token_type_ids=token_id_typ, attention_mask=att_mask, labels = label)
       pred_label = torch.argmax(pred_vector, axis=1)
       pred_label = pred_label + 1
       total_pred_label = torch.cat((total_pred_label, pred_label), axis=0)
     
     total_pred_label = total_pred_label[1:]
     total_pred_label = total_pred_label.detach().cpu().numpy()
     total_pred_label = total_pred_label.tolist()

     output = pd.DataFrame({'Id': ids,
                            'Prediction': total_pred_label})
     output.set_index('Id')
     path = 'result_partB.csv'
     output.to_csv(path, index = False)
     output.head(5)
    
main()


Fetching data...
Fetched and extracted data!


100%|██████████| 231508/231508 [00:00<00:00, 424356.99B/s]


Input IDs and padding generated!
Attention Masks generated!
Total training documents #: 1497
Total Validation documents #: 374
Total Test documents #: 1871
Max sequence Length after padding: 80


  0%|          | 0/3 [00:00<?, ?it/s]


Training completed for epoch 1
Training accuracy for epoch 1: 0.8076152304609219
Training time for this epoch: 88.16158294677734
Total number of training examples: 1497
Number of correctly predicted examples: 1209

Training completed for epoch 1
Validation accuracy for epoch 1: 0.9144385026737968
Validation time for this epoch: 6.465898275375366
Total number of Validation examples: 374
Number of correctly predicted examples: 342


 33%|███▎      | 1/3 [01:38<03:16, 98.37s/it]


Training completed for epoch 2
Training accuracy for epoch 2: 0.897127588510354
Training time for this epoch: 88.14215731620789
Total number of training examples: 1497
Number of correctly predicted examples: 1343


 67%|██████▋   | 2/3 [03:13<01:37, 97.25s/it]


Training completed for epoch 2
Validation accuracy for epoch 2: 0.9144385026737968
Validation time for this epoch: 6.464851379394531
Total number of Validation examples: 374
Number of correctly predicted examples: 342

Training completed for epoch 3
Training accuracy for epoch 3: 0.8931195724782899
Training time for this epoch: 88.10033535957336
Total number of training examples: 1497
Number of correctly predicted examples: 1337



Training completed for epoch 3
Validation accuracy for epoch 3: 0.9144385026737968
Validation time for this epoch: 6.465304136276245
Total number of Validation examples: 374
Number of correctly predicted examples: 342


### _2.csv is giving better results.